In [8]:
# Import libraries
import random
import time
import numpy as np

# Helper/miscellaneous functions
from src.mnist_loader import load_data_wrapper

def sigmoid(z):
    '''The sigmoid function.'''
    return 1.0/(1.0+np.exp(-z))

def dsigmoid(z):
    '''The derivative of the sigmoid function.'''
    return sigmoid(z)*(1-sigmoid(z))

In [10]:
class Network(object): # Inherits from object
    
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        # The biases are added to every layer but the first.
        self.biases = [np.random.randn(x, 1) for x in sizes[1:]] 
        # Weights connect the layers. The first layer's size is the width, the second layer's is the height.
        self.weights = [np.random.randn(x,y) for y,x in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        '''Return the output of the network if ``a`` is input.'''
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

In [11]:
dummy_net = Network([2, 3, 1])
dummy_net.feedforward([2,5])


array([[0.86799368, 0.89793421, 0.87831893]])

In [12]:
print(dummy_net.weights)
print(dummy_net.biases)

[array([[-0.10664109, -0.74266379],
       [-0.74410839,  0.1373196 ],
       [ 0.77185863, -0.72513384]]), array([[0.1871334 , 0.41367726, 0.83710998]])]
[array([[-0.27964209],
       [ 0.51096677],
       [-1.3538927 ]]), array([[1.86318843]])]
